In [1]:
sc.hadoopConfiguration.set("fs.s3n.awsAccessKeyId",sys.env("AWS_ACCESS_KEY"))
sc.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey", sys.env("AWS_SECRET_KEY"))

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.2.10:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1600642335959)
SparkSession available as 'spark'


In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val schema = StructType(
  Array(
    StructField("ip", StringType, true),
    StructField("app", StringType, true),
    StructField("device", StringType, true),
    StructField("os", StringType, true),
    StructField("channel", StringType, true),
    StructField("click_time", StringType, true),
    StructField("attributed_time", StringType, true),
    StructField("is_attributed", StringType, true) 
  )
)

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(ip,StringType,true), StructField(app,StringType,true), StructField(device,StringType,true), StructField(os,StringType,true), StructField(channel,StringType,true), StructField(click_time,StringType,true), StructField(attributed_time,StringType,true), StructField(is_attributed,StringType,true))


In [3]:
// data source: https://www.kaggle.com/matleonard/feature-engineering-data
spark.read.format("csv")
    .schema(schema)
    .load("s3n://renan-test/adtracking/*.csv")
    .limit(10).show()

+------+---+------+---+-------+-------------------+-------------------+-------------+
|    ip|app|device| os|channel|         click_time|    attributed_time|is_attributed|
+------+---+------+---+-------+-------------------+-------------------+-------------+
| 78377|  3|     1| 19|    280|2017-11-07 02:03:23|               null|            0|
| 92057| 14|     1|149|    123|2017-11-07 02:03:23|               null|            0|
| 26726|  3|     1| 19|    280|2017-11-07 02:03:23|               null|            0|
|114083|  9|     1| 22|    134|2017-11-07 02:03:23|               null|            0|
|115621| 35|     1| 19|     21|2017-11-07 02:03:23|2017-11-07 02:26:17|            1|
|130792| 15|     1| 18|    245|2017-11-07 02:03:23|               null|            0|
|100275| 15|     1| 19|      3|2017-11-07 02:03:23|               null|            0|
|126918| 19|     0| 24|    213|2017-11-07 02:03:23|2017-11-07 11:24:28|            1|
| 98365| 18|     1| 13|    107|2017-11-07 02:03:23|   

In [4]:
val trackingStream =  spark.readStream.format("csv")
    .schema(schema)
    .load("s3n://renan-test/adtracking/*.csv")
    .withColumn("value",to_json(struct($"ip",$"app",$"device",$"os",$"channel",$"click_time",$"attributed_time",$"is_attributed")))
    .withColumn("key",$"app")
    .select("key","value")

trackingStream.writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("topic", "tracking")
    .option("checkpointLocation", "/home/rhuanca/demo/_checkpoints/tracking")
    .start()
    .awaitTermination()

org.apache.spark.sql.streaming.StreamingQueryException:  Query [id = c9b8dd27-4d0c-4577-bea7-428e228e1296, runId = c5611f4e-9e9e-44b8-92c4-fe244b52edb5] terminated with exception: Read timed out